In [65]:

import json
import pandas as pd
import numpy as np
from toolz import memoize
import datetime
import math

from tqdm import tqdm
from src.utils.mouselab_jas import MouselabJas
from src.utils.distributions import Normal, expectation
from src.utils.env_creation import create_tree, create_init
from src.utils.env_export import create_json
from src.utils.data_classes import MouselabConfig, Action
from simulation import run_simulation
from src.policy.jas_voc_policy import JAS_voc_policy
from src.policy.jas_policy import RandomPolicy, ExhaustivePolicy, RandomNPolicy
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from src.utils.utils import sigma_to_tau
import numpy as np
from src.utils.env_export import format_payoff


sns.set(rc={'figure.figsize':(10,6)})
sns.set(font_scale=1.5)
sns.set_theme()

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
data = json.load(open("./data/dataclips_ancljubojkwcxdlizfihpmhkuogm.json"))

In [45]:
language_index = data["fields"].index("language")
response_data_index = data["fields"].index("datastring")
begin_index = data["fields"].index("beginhit")
end_index = data["fields"].index("endhit")

f = '%Y-%m-%d %H:%M:%S.%f'

In [46]:
# Fixed from paper
num_projects = 5
num_criteria = 6
weights = [0.0206795, 0.0672084, 0.2227102, 0.1067428, 0.4665054, 0.1161537]
expert_stds = [1.5616618964384956, 1.449172525995787, 1.5205992970609392, 1.5469422429523034, 1.511270787760881, 1.455189251463794]
expert_taus = list(sigma_to_tau(np.array(expert_stds)))

mu = [3.6, 3.1666666666666665, 3.6, 3.1333333333333333, 3.6666666666666665, 2.3]
sigma = [1.3544307876819288, 1.2617266038997932, 1.3796551293211172, 1.2521246311585852, 1.5161960871578068, 0.9523111632886272]

init = create_init([0]+(mu*num_projects), [0]+(sigma*num_projects))
expert_costs = [0.002]*6

config = MouselabConfig(num_projects, num_criteria, expert_costs, expert_taus, init, criteria_scale=weights, term_belief=False)
env = MouselabJas(config=config)

In [47]:
def convert_action(project: int, criteria: int, expert: int, config: MouselabConfig) -> Action:
    query = (1 + criteria) + project*config.num_criterias
    action = Action(expert=expert, query=query)
    return action

In [82]:
df_index = ["Participant", "TrialId", "Score", "ExpectedScore", "NumClicks", "Actions", "Selection",
        "Seed"]    
df_data = []

bonus_data = {}
known_workers = []
good_responses = 0
demographics = []

# Parse raw mturk data into dataframe
for p_index, p_data in tqdm(enumerate(data["values"])):
    # Filter out empty responses
    language = p_data[language_index]
    response_data = p_data[response_data_index]
    if p_data[begin_index] and p_data[end_index]:
        begin = datetime.datetime.strptime(p_data[begin_index], f)
        end = datetime.datetime.strptime(p_data[end_index], f)
        duration = (end - begin).total_seconds()
    else:
        duration = None
    if response_data != None:
        p_res_obj = json.loads(response_data)
        condition = p_res_obj["condition"]
        worker = p_index #p_res_obj["workerId"]# 
        if worker in known_workers:
            print("Duplicate worker", worker)
        else: 
            known_workers.append(worker)
        p_res = p_res_obj["data"]
        participant_responses = []
        if "quiz_failures" in p_res_obj["questiondata"].keys():
            quiz_failures = p_res_obj["questiondata"]["quiz_failures"]
        else:
            quiz_failures = 0
        if "final_bonus" in p_res_obj["questiondata"].keys():
            bonus =  p_res_obj["questiondata"]["final_bonus"]
        else:
            bonus = 0
        participant_survey = {"Participant": worker, "Condition": condition, "Language": language, "QuizAttempts": 0, "QuizFailures": quiz_failures, "Bonus": bonus, "Duration": duration}
        completed_test_trials = sum([1 if ('trial_id' in  p_res[i]['trialdata'].keys() and p_res[i]['trialdata']['trial_id'].startswith("test")) else 0 for i in range(len(p_res))])
        if (quiz_failures < 3) and (completed_test_trials == 10):
            for i in range(len(p_res)):
                # Get test trials
                if 'trial_id' in p_res[i]['trialdata'].keys() and p_res[i]['trialdata']['trial_id'].startswith("test"):
                    trial_id = p_res[i]['trialdata']['trial_id']
                    seed = int(p_res[i]['trialdata']['seed'])
                    ground_truth = p_res[i]['trialdata']['ground_truth']
                    # project, criteria, expert
                    clicks = p_res[i]['trialdata']["clicks"]
                    num_clicks = len(clicks)
                    selected_project = p_res[i]['trialdata']["selected_project"]
                    term_reward = p_res[i]['trialdata']["reward"]
                    expected_reward = p_res[i]['trialdata']["expected_reward"]
                    # Calculate real env rewards
                    env.reset(seed=seed)
                    actions = [convert_action(*click, config) for click in clicks]
                    cost = 0
                    for action in actions:
                        _, reward, _, _ = env.step(action)
                        cost += reward
                    path = np.array(range(1, config.num_criterias+1))+(selected_project*config.num_criterias)
                    env_expected_reward = cost + env.expected_path_value(path, env.state)
                    env_term_reward = cost + env.path_value(path)
                    assert np.all(np.isclose(ground_truth, env.ground_truth.tolist()))
                    assert np.all(np.isclose(np.array(format_payoff(config.num_projects, config.num_criterias, env.expert_truths.tolist())), p_res[i]['trialdata']['payoff_matrix']))
                    assert np.isclose(term_reward, env_term_reward)
                    assert np.isclose(expected_reward, env_expected_reward)
                    
                    #["Participant", "TrialId", "Score", "ExpectedScore", "NumClicks", "Actions", "Selection", "Seed"] 
                    df_data.append([worker, trial_id, term_reward, expected_reward, num_clicks, clicks, selected_project, seed])

df = pd.DataFrame(df_data, columns=df_index)
print("Responses:", len(df["Participant"].unique()))

8it [00:00, 82.70it/s]

Responses: 5


In [83]:
participant_scores = df.groupby("Participant").agg(["mean", "std"])

C:\Users\lheindrich\AppData\Local\Temp\ipykernel_19540\1370106099.py:1: FutureWarning: ['TrialId', 'Actions'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  participant_scores = df.groupby("Participant").agg(["mean", "std"])


In [84]:
participant_scores

Score           ExpectedScore           NumClicks            \
                 mean       std          mean       std      mean       std   
Participant                                                                   
0            3.983642  0.554479      3.739988  0.090342      15.8  2.201010   
4            4.002598  0.288937      3.686120  0.200170      31.9  3.314949   
5            3.820292  0.628682      3.604823  0.225885       4.0  1.563472   
6            3.041022  0.869564      3.401164  0.000000       0.0  0.000000   
7            3.870474  0.471494      3.730077  0.065036      10.6  6.149977   

            Selection            Seed           
                 mean       std  mean      std  
Participant                                     
0                 1.9  1.286684  15.5  3.02765  
4                 2.2  1.032796  15.5  3.02765  
5                 1.1  0.875595  15.5  3.02765  
6                 0.6  0.516398  15.5  3.02765  
7                 2.1  1.663330  15.5  3.02765

In [85]:
participant_scores = participant_scores.sort_values(("Score", "mean"), ascending=False)
participant_scores

Score           ExpectedScore           NumClicks            \
                 mean       std          mean       std      mean       std   
Participant                                                                   
4            4.002598  0.288937      3.686120  0.200170      31.9  3.314949   
0            3.983642  0.554479      3.739988  0.090342      15.8  2.201010   
7            3.870474  0.471494      3.730077  0.065036      10.6  6.149977   
5            3.820292  0.628682      3.604823  0.225885       4.0  1.563472   
6            3.041022  0.869564      3.401164  0.000000       0.0  0.000000   

            Selection            Seed           
                 mean       std  mean      std  
Participant                                     
4                 2.2  1.032796  15.5  3.02765  
0                 1.9  1.286684  15.5  3.02765  
7                 2.1  1.663330  15.5  3.02765  
5                 1.1  0.875595  15.5  3.02765  
6                 0.6  0.516398  15.5  3.02765

In [86]:
sorted_participants = participant_scores.index.tolist()
half_participants = math.ceil(len(sorted_participants)/2)
high_bonus = sorted_participants[:half_participants]
low_bonus = sorted_participants[half_participants:]
for id in high_bonus:
    print(f"{id},0.75")
for id in low_bonus:
    print(f"{id},0.25")

4,0.75
0,0.75
7,0.75
5,0.25
6,0.25
